## 문제2 백화점 고객의 1년 간 구매 데이터를 활용해

- 데이터 전처리
- Feature Engineering
- 모델링 (분류 알고리즘 사용)
- 하이퍼파라미터 튜닝 (초매개변수 최적화)
- 모형 앙상블
- csv제출

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np

In [3]:
x = pd.read_csv('/content/drive/MyDrive/빅데이터 분석기사/data/department_data/X_train.csv' , encoding = 'euc-kr' )
y = pd.read_csv('/content/drive/MyDrive/빅데이터 분석기사/data/department_data/y_train.csv' , encoding = 'euc-kr')
test = pd.read_csv('/content/drive/MyDrive/빅데이터 분석기사/data/department_data/X_test.csv' , encoding = 'euc-kr')

In [4]:
## 간단한 EDA
x.shape , y.shape , test.shape

((3500, 10), (3500, 2), (2482, 10))

In [5]:
x.describe()

,cust_id,총구매액,최대구매액,환불금액,내점일수,내점당구매건수,주말방문비율,구매주기
count,3500.000000,3.500000e+03,3.500000e+03,1.205000e+03,3500.000000,3500.000000,3500.000000,3500.000000
mean,1749.500000,9.191925e+07,1.966424e+07,2.407822e+07,19.253714,2.834963,0.307246,20.958286
std,1010.507298,1.635065e+08,3.199235e+07,4.746453e+07,27.174942,1.912368,0.289752,24.748682
min,0.000000,-5.242152e+07,-2.992000e+06,5.600000e+03,1.000000,1.000000,0.000000,0.000000
25%,874.750000,4.747050e+06,2.875000e+06,2.259000e+06,2.000000,1.666667,0.027291,4.000000
50%,1749.500000,2.822270e+07,9.837000e+06,7.392000e+06,8.000000,2.333333,0.256410,13.000000
75%,2624.250000,1.065079e+08,2.296250e+07,2.412000e+07,25.000000,3.375000,0.448980,28.000000
max,3499.000000,2.323180e+09,7.066290e+08,5.637530e+08,285.000000,22.083333,1.000000,166.000000


In [6]:
x.head()

,cust_id,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
0,0,68282840,11264000,6860000.0,기타,강남점,19,3.894737,0.527027,17
1,1,2136000,2136000,300000.0,스포츠,잠실점,2,1.500000,0.000000,1
2,2,3197000,1639000,NaN,남성 캐주얼,관악점,2,2.000000,0.000000,1
3,3,16077620,4935000,NaN,기타,광주점,18,2.444444,0.318182,16
4,4,29050000,24000000,NaN,보석,본 점,2,1.500000,0.000000,85


In [7]:
y

,cust_id,gender
0,0,0
1,1,0
2,2,1
3,3,1
4,4,0
...,...,...
3495,3495,1
3496,3496,1
3497,3497,0
3498,3498,0


In [8]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   cust_id  3500 non-null   int64  
 1   총구매액     3500 non-null   int64  
 2   최대구매액    3500 non-null   int64  
 3   환불금액     1205 non-null   float64
 4   주구매상품    3500 non-null   object 
 5   주구매지점    3500 non-null   object 
 6   내점일수     3500 non-null   int64  
 7   내점당구매건수  3500 non-null   float64
 8   주말방문비율   3500 non-null   float64
 9   구매주기     3500 non-null   int64  
dtypes: float64(3), int64(5), object(2)
memory usage: 273.6+ KB


In [9]:
x.isnull().sum()

cust_id       0
총구매액          0
최대구매액         0
환불금액       2295
주구매상품         0
주구매지점         0
내점일수          0
내점당구매건수       0
주말방문비율        0
구매주기          0
dtype: int64

In [10]:
test.isnull().sum()

cust_id       0
총구매액          0
최대구매액         0
환불금액       1611
주구매상품         0
주구매지점         0
내점일수          0
내점당구매건수       0
주말방문비율        0
구매주기          0
dtype: int64

### 데이터 전처리

In [11]:
## 결측치 처리
x = x.fillna(0)
test = test.fillna(0)

display(x.isnull().sum())
display(test.isnull().sum())

cust_id    0
총구매액       0
최대구매액      0
환불금액       0
주구매상품      0
주구매지점      0
내점일수       0
내점당구매건수    0
주말방문비율     0
구매주기       0
dtype: int64

cust_id    0
총구매액       0
최대구매액      0
환불금액       0
주구매상품      0
주구매지점      0
내점일수       0
내점당구매건수    0
주말방문비율     0
구매주기       0
dtype: int64

In [12]:
x = x.drop(['cust_id'] , axis = 1)
cust_id = test.pop('cust_id')

## feature engineering


In [13]:
from sklearn.preprocessing import LabelEncoder

cols = ['주구매상품' , '주구매지점']
for col in cols:
    le = LabelEncoder()
    x[col] = le.fit_transform(x[col])
    test[col] = le.fit_transform(test[col])

x.head()

,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
0,68282840,11264000,6860000.0,5,0,19,3.894737,0.527027,17
1,2136000,2136000,300000.0,21,19,2,1.500000,0.000000,1
2,3197000,1639000,0.0,6,1,2,2.000000,0.000000,1
3,16077620,4935000,0.0,5,2,18,2.444444,0.318182,16
4,29050000,24000000,0.0,15,8,2,1.500000,0.000000,85


In [14]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   총구매액     3500 non-null   int64  
 1   최대구매액    3500 non-null   int64  
 2   환불금액     3500 non-null   float64
 3   주구매상품    3500 non-null   int64  
 4   주구매지점    3500 non-null   int64  
 5   내점일수     3500 non-null   int64  
 6   내점당구매건수  3500 non-null   float64
 7   주말방문비율   3500 non-null   float64
 8   구매주기     3500 non-null   int64  
dtypes: float64(3), int64(6)
memory usage: 246.2 KB


### 모델링 + 하이퍼 파리미터 튜닝

In [15]:
from sklearn.ensemble import RandomForestClassifier


model = RandomForestClassifier(max_depth = 5 , n_estimators = 100 , random_state = 42)
model.fit(x,y['gender'])

RandomForestClassifier(max_depth=5, random_state=42)

In [16]:
model.score(x,y['gender'])

0.6911428571428572

In [17]:
y_pred = model.predict_proba(test)[:,1]

In [18]:
df = pd.DataFrame({'cust_id' :cust_id , 'gender':y_pred})

In [19]:
df.head()

,cust_id,gender
0,3500,0.397012
1,3501,0.178500
2,3502,0.215828
3,3503,0.425004
4,3504,0.468121


In [20]:
df.to_csv('result_predict.csv', index = False) # index False 옵션을 줘야함


In [21]:
pd.read_csv('result_predict.csv')

,cust_id,gender
0,3500,0.397012
1,3501,0.178500
2,3502,0.215828
3,3503,0.425004
4,3504,0.468121
...,...,...
2477,5977,0.467473
2478,5978,0.524917
2479,5979,0.434257
2480,5980,0.374825
